# Quantum CSD Compiling Intro

The purpose of this notebook is to give a quick introduction to Qubiter's 
CSD quantum compiler capabilities. 

By a quantum complier, we mean
a computer program that takes as input an arbitrary unitary matrix $U$ of dimension $N=2^n$
and returns a SEO (sequence of elementary operations, in this case CNOTs and single qubit
rotations) that equals $U$. There are various kinds of quantum 
compilers. Suppose $U$ is of the form $U=e^{-itH}$, where $t$ is time and $H$ is
the Hamiltonian operator. If $H$ has a form which is known a priori, a situation
that is common in Physics and Chemistry, then a popular way of expanding $U$
is by using the Trotter-Suzuki approximation. If the form of $H$ is not
known a priori as is common in Artificial Intelligence, then
we recommend using the CS (Cosine-Sine) decomposition of Linear Algebra.
Both methods are already implemented in Qubiter, but this notebook is about
the CSD.

Doing CSD quantum compiling with Qubiter requires using the classes in the quantum_CSD_compiler
folder, which will only work properly if you install, besides all the Qubiter
Python files and a Python distro that includes numpy and scipy (for example, Anaconda),
some binary libraries prepared by Artiste-q.net which include
a Python wrapper for a LAPACK subroutine
called cuncsd.f that performs CSDs. How to install those binary libraries
is explained elsewhere in this site. Henceforth, we will assume 
all the necessary files have been installed on your computer if you want to redo the calculations.

The quantum_CSD_compiler folder includes a pdf called csd-intro.pdf that gives
an introduction to the CSD. 

Some external references:


1. R.R. Tucci, A Rudimentary Quantum Compiler(2cnd Ed.)
    https://arxiv.org/abs/quant-ph/9902062

2. Qubiter 1.11, a C++ program whose first version was released together
    with Ref.1 above. Qubiter 1.11 is included in the
    quantum_CSD_compiler/LEGACY folder of this newer, pythonic version of Qubiter
    
3. R.R. Tucci, Quantum Fast Fourier Transform Viewed as a Special Case of Recursive Application of Cosine-Sine Decomposition, https://arxiv.org/abs/quant-ph/0411097



Qubiter applies CSD recursively
to build a tree of node matrices. The product of those node matrices,
if read in the correct order, is equal to the input matrix $U$.

As an example, let us use for $U$ a 3 qubit quantum Fourier matrix.
We can create an object of class Tree with $U$ 
as input as follows

In [1]:
import os
import sys
print(os.getcwd())
os.chdir('../../')
print(os.getcwd())
sys.path.insert(0,os.getcwd())
%load_ext autoreload
%autoreload 2

/Users/yaroslav/Dropbox/Turation/fork_qubiter/qubiter/qubiter/quantum_CSD_compiler
/Users/yaroslav/Dropbox/Turation/fork_qubiter/qubiter


In [2]:
import numpy as np
import math
from scipy.stats import unitary_group

# Qubiter libs
from qubiter.FouSEO_writer import *
from qubiter.CGateSEO_writer import *
from qubiter.CGateExpander import *
from qubiter.quantum_CSD_compiler.MultiplexorExpander import *
from qubiter.quantum_CSD_compiler.Tree import *
from qubiter.quantum_CSD_compiler.DiagUnitarySEO_writer import *
from qubiter.quantum_CSD_compiler.MultiplexorSEO_writer import *
from qubiter.quantum_CSD_compiler.DiagUnitaryExpander import *
from qubiter.device_specific.Qubiter_to_AnyQasm import *
import pandas as pd

loaded OneBitGates, WITHOUT autograd.numpy


In [3]:
num_bits = 2
init_unitary_mat = unitary_group.rvs(2**num_bits)
emb = CktEmbedder(num_bits, num_bits)
file_prefix = "decomposition_demo"
t = Tree(True, file_prefix, emb, init_unitary_mat, verbose=False)
t.close_files()

/usr/local/lib/python3.7/site-packages/pycsd.py:114: ComplexWarning: Casting complex values to real discards the imaginary part
  signs=signs)


The above code automatically creates an expansion of $U$ 
into DIAG and MP_Y lines. Next we print the Picture file that was created.

In [4]:
file = './qubiter/quantum_CSD_compiler/io_folder/'+file_prefix + "_2_ZLpic.txt"
#file = './qubiter/io_folder/'+file_prefix + "_2_ZLpic.txt"
#file = './'+file_prefix + "_2_ZLpic.txt"
df = pd.read_csv(file, delim_whitespace=True, header=None)

In [5]:
style = 'exact'
xer = MultiplexorExpander(file_prefix, num_bits, style, verbose=False)

In [6]:
xer = DiagUnitaryExpander(file_prefix+'_X1', num_bits, style)

In [7]:
xer = MultiplexorExpander(file_prefix+'_X2', num_bits, style, verbose=False)

In [8]:
exp = CGateExpander(file_prefix+'_X3', num_bits)

In [10]:
file = './qubiter/quantum_CSD_compiler/io_folder/'+file_prefix + '_X4_2_ZLpic.txt'
log = open(file)
for line in log:
    print(line)

|   Ph  

|   Rz  

@---X   

|   Rz  

@---X   

Rz  |   

|   Ry  

@---X   

|   Ry  

@---X   

|   Ph  

|   Rz  

@---X   

|   Rz  

@---X   

Rz  |   

Ry  |   

X---@   

Ry  |   

X---@   

|   Ph  

|   Rz  

@---X   

|   Rz  

@---X   

Rz  |   

|   Ry  

@---X   

|   Ry  

@---X   

|   Ph  

|   Rz  

@---X   

|   Rz  

@---X   

Rz  |   



Let us also print the corresponding English file that was created.

In [11]:
file = './qubiter/quantum_CSD_compiler/io_folder/'+file_prefix + '_X4_2_eng.txt'
log = open(file)
for line in log:
    print(line)

PHAS	51.79475926672995	AT	0

ROTZ	-51.79475926672993	AT	0

SIGX	AT	0	IF	1T

ROTZ	31.295448060229056	AT	0

SIGX	AT	0	IF	1T

ROTZ	-31.295448060229056	AT	1

ROTY	34.96920092413325	AT	0

SIGX	AT	0	IF	1T

ROTY	-3.8248697377192262	AT	0

SIGX	AT	0	IF	1T

PHAS	-29.653934501764027	AT	0

ROTZ	51.39640585314514	AT	0

SIGX	AT	0	IF	1T

ROTZ	-51.39640585314514	AT	0

SIGX	AT	0	IF	1T

ROTZ	29.653934501764017	AT	1

ROTY	44.28100221833246	AT	1

SIGX	AT	1	IF	0T

ROTY	-18.26012216643267	AT	1

SIGX	AT	1	IF	0T

PHAS	-55.54568658569963	AT	0

ROTZ	55.545686585699606	AT	0

SIGX	AT	0	IF	1T

ROTZ	-0.3990090418688226	AT	0

SIGX	AT	0	IF	1T

ROTZ	0.3990090418688226	AT	1

ROTY	24.107029551480597	AT	0

SIGX	AT	0	IF	1T

ROTY	-2.851964242429068	AT	0

SIGX	AT	0	IF	1T

PHAS	-56.595138179266314	AT	0

ROTZ	-42.34095030037969	AT	0

SIGX	AT	0	IF	1T

ROTZ	19.754557586994284	AT	0

SIGX	AT	0	IF	1T

ROTZ	-84.3191778807418	AT	1



In [12]:
import qubiter.device_specific.chip_couplings_ibm as ibm
from qubiter.device_specific.Qubiter_to_IBMqasm import Qubiter_to_IBMqasm

aqasm_name = 'IBMqasm'
c_to_tars = None # ibm.ibmq5YorktownTenerife_c_to_tars
Qubiter_to_IBMqasm(file_prefix + '_X4', num_bits, aqasm_name=aqasm_name,
                   c_to_tars=c_to_tars, write_qubiter_files=True)